Script that 
* takes as input the files config-layers-X (X: polygon, point, linestring), and municipality codes
* for each X,
    * determine evaluation layers
    * fetch and merge gdfs for given muni (if they exist)
    * save to `input-for-bike-node-planner/X`

In [1]:
# import libraries
import os
import yaml
import geopandas as gpd
import pandas as pd

In [2]:
# read config files
config = yaml.load(
    open("../config.yml"), 
    Loader=yaml.FullLoader)
proj_crs = config["proj_crs"]

municipalities = yaml.load(
    open("../config-municipalities.yml"), 
    Loader=yaml.FullLoader)
codes = municipalities["kommunekode"]

geomtypes = ["point", "linestring", "polygon"]
config_layers = {}
for geomtype in geomtypes:
    config_layers[geomtype] = yaml.load(
        open(f"../config-layers-{geomtype}.yml"), 
        Loader=yaml.FullLoader)

# todo: create "study area" polygon

In [3]:
layer_dict = {}

for geomtype in geomtypes:

    layer_dict[geomtype] = {}

    # determine evaluation layers
    layers = []
    for v in config_layers[geomtype].values():
        layers += list(set(v.values()))
    layers = list(set(layers))
    
    # determine data sets that go into each layer

    # key is name of merged output layer, value is a dict 
    for layer in layers:
        layer_dict[geomtype][layer] = {}
    
    # adding data source as key to dictindict IF relevant to layer
    for datasource, vdict in config_layers[geomtype].items():
        for layer in (set(vdict.values())):
            layer_dict[geomtype][layer][datasource] = []
    
    for datasource, vdict in config_layers[geomtype].items():
        for k, v in vdict.items():
            layer_dict[geomtype][v][datasource] += [k]

for geomtype in geomtypes:
    if "ignore" in layer_dict[geomtype]:
        del layer_dict[geomtype]["ignore"]

print(layer_dict)


{'point': {'service': {'facilit_indkoeb': ['supermarked', 'bager', 'kiosk', 'slagter', 'Lokale smagsoplevelser', 'tankstation', 'fisk'], 'facilit_overnatning': ['Bed & Breakfast', 'Hotel', 'Campingplads', 'Slot/Herregård', 'Vandrehjem', 'Feriecenter', 'Kro'], 'facilit_service': ['Turistkontor']}, 'poi': {'land_besoegs': ['Museum', 'Forlystelsespark', 'Dyrepark, zoo', 'Dyrepark', 'Anden seværdighed', 'Frilandsmuseum', 'Aboret, blomsterpark o.l.'], 'land_fortid': ['Gravhøj', 'Vold eller voldsted', 'Langdysse', 'Jættestue', 'Fortidsminde/ruin', 'Skanse', 'Ruin', 'Batteri', 'Dysse', 'Runddysse', 'Andet historisk anlæg', 'Fæstningsanlæg', 'Oldtidsvej', 'Vikingeborg', 'Oldtidsminde'], 'land_landemaerke': ['Kirke', 'Fyrtårn', 'Vejrmølle', 'Herregård', 'Slot'], 'land_udflugt': ['Udsigtspunkt', 'Naturområde', 'Fugletårn/Udsigtstårn', 'Badestrand/-sted', 'Blå Flag strand', 'Badepunkt', 'Badevandsflag']}, 'facility': {'facilit_rasteplads': ['Bænk', 'Picnicbord', 'Legeplads', 'Borde-bænke/Picnic',

In [4]:
for geomtype in geomtypes:

    print(geomtype)

    for layername, datadict in layer_dict[geomtype].items():
    
        print("\t", layername)
        final_gdf = gpd.GeoDataFrame()
        
        for k, v in datadict.items():

            gdf = gpd.GeoDataFrame()

            for code in codes:
                print("\t \t", code, k)
                # for each code, check if file exists, if yes: read it in, if not empty: concatenate
                fp = f"../data/{geomtype}/{code}/{k}.gpkg"
                if os.path.exists(fp):
                    gdf_muni = gpd.read_file(fp)
                    if not gdf_muni.empty:
                        gdf = pd.concat(
                            [
                                gdf,
                                gdf_muni
                            ]
                        )

            final_gdf = pd.concat(
                [
                    final_gdf,
                    gdf[gdf["type"].isin(v)]
                ]
            )

        if not final_gdf.empty:
            final_gdf = final_gdf.reset_index(drop=True)
            final_gdf.to_file(
                f"../input-for-bike-node-planner/{geomtype}/{layername}.gpkg", 
                index = False
            )
            print("\t", f"{layername} saved")
        else:
            print("\t", f"No data found for layer {layername}")

point
	 service
	 	 0259 facilit_indkoeb
	 	 0320 facilit_indkoeb
	 	 0329 facilit_indkoeb
	 	 0336 facilit_indkoeb
	 	 0370 facilit_indkoeb
	 	 0259 facilit_overnatning
	 	 0320 facilit_overnatning
	 	 0329 facilit_overnatning
	 	 0336 facilit_overnatning
	 	 0370 facilit_overnatning
	 	 0259 facilit_service
	 	 0320 facilit_service
	 	 0329 facilit_service
	 	 0336 facilit_service
	 	 0370 facilit_service
	 service saved
	 poi
	 	 0259 land_besoegs
	 	 0320 land_besoegs
	 	 0329 land_besoegs
	 	 0336 land_besoegs
	 	 0370 land_besoegs
	 	 0259 land_fortid
	 	 0320 land_fortid
	 	 0329 land_fortid
	 	 0336 land_fortid
	 	 0370 land_fortid
	 	 0259 land_landemaerke
	 	 0320 land_landemaerke
	 	 0329 land_landemaerke
	 	 0336 land_landemaerke
	 	 0370 land_landemaerke
	 	 0259 land_udflugt
	 	 0320 land_udflugt
	 	 0329 land_udflugt
	 	 0336 land_udflugt
	 	 0370 land_udflugt
	 poi saved
	 facility
	 	 0259 facilit_rasteplads
	 	 0320 facilit_rasteplads
	 	 0329 facilit_rasteplads
	 	 0